In [1]:
.libPaths(R.home('/opt/homebrew/Caskroom/miniforge/base/lib/R/library'))
Output = '/Users/alexis/Documents/R/Engineering the Future/Output'
cur_date = '081825'

library(readxl)
library(tidyverse)
library(reshape2)
library(openxlsx)

# reading in files
participant_df = data.frame(read_excel("Input/Participant Information.xlsx"))[,1:7]
institution_df = data.frame(read_excel("Input/Institution Information.xlsx"))

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.2     ✔ tibble    3.3.0
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.1.0     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths




In [2]:
head(participant_df)
head(institution_df)

,Year_of_Attendance,First_Name,Last_Name,Full_Name,Institution,Department,Position
,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,2001,Jalal,Abedi,Jalal Abedi,Clark Atlanta University,Chemical Engineering,Assistant Professor
2,2001,Lukee,Achenie,Lukee Achenie,University of Connecticut,Chemical Engineering,Associate Professor
3,2001,Wudneh,Admassu,Wudneh Admassu,University of Idaho,Chemical Engineering,Professor and Department Chair
4,2001,Pedro,Alvarez,Pedro Alvarez,University of Iowa,Civil & Environmental Engineering,Associate Professor
5,2001,Camille,Anderson,Camille Anderson,University of California - Berkeley,Chemical Engineering,PhD Candidate
6,2001,Lynden,Archer,Lynden Archer,Cornell University,Chemical Engineering,Associate Professor


,Institution,Type,Region
,<chr>,<chr>,<chr>
1,Arizona State University - Tempe Campus,PWI,SW
2,Arizona State University ‐ Polytechnic Campus,PWI,SW
3,Armstrong Atlantic State University,HSI,SE
4,Auburn University,PWI,SE
5,Battelle Pacific Northwest Laboratory,Lab,NW
6,Baylor College of Medicine,PWI,SW


## 1. How many participants returned and which years did they attend?

In [3]:
repeating_part_df = participant_df %>%
    group_by(Full_Name) %>%
    count() %>%
    filter(n > 1)

head(repeating_part_df)

Full_Name,n
<chr>,<int>
Addle Hicks,2
Alfred Cruz,2
Anjelica Gonzalez,3
Asegun Henry,2
Carmen Lilley,2
Cheryl Gomillion,2


In [4]:
repeating_names = repeating_part_df$Full_Name
length(repeating_names)

repeat_year_df = participant_df[,c(1,4)] %>%
    filter(Full_Name %in% repeating_names) %>%
    arrange(Full_Name)

head(repeat_year_df)

[1] 58

,Year_of_Attendance,Full_Name
,<dbl>,<chr>
1,2006,Addle Hicks
2,2009,Addle Hicks
3,2010,Alfred Cruz
4,2016,Alfred Cruz
5,2004,Anjelica Gonzalez
6,2006,Anjelica Gonzalez


58 participants attended at least twice.

## 2. How many participants represented each department for each year (i.e., chemistry engineering for 2001 had 5 participants)?

In [5]:
full_df = inner_join(participant_df, institution_df)

head(full_df)

Joining with `by = join_by(Institution)`


,Year_of_Attendance,First_Name,Last_Name,Full_Name,Institution,Department,Position,Type,Region
,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,2001,Jalal,Abedi,Jalal Abedi,Clark Atlanta University,Chemical Engineering,Assistant Professor,HBCU,SE
2,2001,Lukee,Achenie,Lukee Achenie,University of Connecticut,Chemical Engineering,Associate Professor,PWI,NE
3,2001,Wudneh,Admassu,Wudneh Admassu,University of Idaho,Chemical Engineering,Professor and Department Chair,PWI,W
4,2001,Pedro,Alvarez,Pedro Alvarez,University of Iowa,Civil & Environmental Engineering,Associate Professor,PWI,MW
5,2001,Camille,Anderson,Camille Anderson,University of California - Berkeley,Chemical Engineering,PhD Candidate,AANAPISI/HSI,W
6,2001,Lynden,Archer,Lynden Archer,Cornell University,Chemical Engineering,Associate Professor,PWI,NE


In [6]:
multiple_dept_df = full_df %>%
    # splitting up the department col for people listed in multiple departments
    filter(grepl(",", Department)) %>%
    separate(Department, c("Dept1", "Dept2", "Dept3"), sep = ",") %>%
    pivot_longer(cols = starts_with("Dept"), values_to = "Department") %>%
    select(-name) %>%
    # removing rows without a 3rd department affiliation
    filter(!is.na(Department))

single_dept_df = full_df %>%
    # splitting up the department col for people listed in multiple departments
    filter(!grepl(",", Department))

recombined_df = rbind(multiple_dept_df, single_dept_df)
head(recombined_df)

Warning message:
“Expected 3 pieces. Missing pieces filled with `NA` in 21 rows [1, 2, 3, 4, 5,
7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, ...].”


Year_of_Attendance,First_Name,Last_Name,Full_Name,Institution,Position,Type,Region,Department
<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
2001,Herek,Clack,Herek Clack,Illinois Institute of Technology,Assistant Professor,PWI,MW,Mechanical & Aerospace Engineering
2001,Herek,Clack,Herek Clack,Illinois Institute of Technology,Assistant Professor,PWI,MW,Material Science Engineering
2001,Norman,Loney,Norman Loney,New Jersey Institute of Technology,Associate Professor,PWI,NE,Chemical Engineering
2001,Norman,Loney,Norman Loney,New Jersey Institute of Technology,Associate Professor,PWI,NE,Environmental Engineering
2001,Howard,Matthew,Howard Matthew,Wayne State University,Associate Professor,PWI,MW,Chemical Engineering
2001,Howard,Matthew,Howard Matthew,Wayne State University,Associate Professor,PWI,MW,Material Science Engineering


In [7]:
dept_counts_df = recombined_df %>%
    # removing people without departmental affliations recorded
    filter(Department != "NA") %>%
    group_by(Year_of_Attendance, Department) %>%
    summarize(n = n()) %>%
    arrange(-n, .by_group = TRUE)

head(dept_counts_df)

`summarise()` has grouped output by 'Year_of_Attendance'. You can override
using the `.groups` argument.


Year_of_Attendance,Department,n
<dbl>,<chr>,<int>
2001,Chemical Engineering,39
2001,Civil & Environmental Engineering,5
2001,Chemical & Nuclear Engineering,4
2001,Material Science Engineering,3
2001,Civil Engineering,2
2001,Advanced Materials & Processing Branch,1


## 3. How many participants and institutions are from each region per year?

In [8]:
regional_part_df = full_df %>%
    group_by(Year_of_Attendance, Region) %>%
    summarize(n = n()) %>%
    arrange(-n, .by_group = TRUE)

head(regional_part_df)

`summarise()` has grouped output by 'Year_of_Attendance'. You can override
using the `.groups` argument.


Year_of_Attendance,Region,n
<dbl>,<chr>,<int>
2001,SE,20
2001,NE,19
2001,MW,14
2001,SW,5
2001,W,4
2004,SE,35


In [9]:
regional_inst_df = unique(full_df[,c(1,5,9)]) %>%
    group_by(Year_of_Attendance, Region) %>%
    summarize(n = n()) %>%
    arrange(-n, .by_group = TRUE)

head(regional_inst_df)

`summarise()` has grouped output by 'Year_of_Attendance'. You can override
using the `.groups` argument.


Year_of_Attendance,Region,n
<dbl>,<chr>,<int>
2001,NE,15
2001,SE,13
2001,MW,11
2001,SW,4
2001,W,3
2004,SE,19


## 4. How many participants and institutions are from each institutional type per year?

In [10]:
final_df = recombined_df %>%
    # splitting up the type col for people at institutions with multiple types
    separate(Type, c("Type1", "Type2"), sep = "/") %>%
    pivot_longer(cols = starts_with("Type"), values_to = "Type") %>%
    select(-name) %>%
    # removing rows without a 2nd type
    filter(!is.na(Type))

head(final_df)

Warning message:
“Expected 2 pieces. Missing pieces filled with `NA` in 502 rows [1, 2, 3, 4, 5,
6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, ...].”


Year_of_Attendance,First_Name,Last_Name,Full_Name,Institution,Position,Region,Department,Type
<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
2001,Herek,Clack,Herek Clack,Illinois Institute of Technology,Assistant Professor,MW,Mechanical & Aerospace Engineering,PWI
2001,Herek,Clack,Herek Clack,Illinois Institute of Technology,Assistant Professor,MW,Material Science Engineering,PWI
2001,Norman,Loney,Norman Loney,New Jersey Institute of Technology,Associate Professor,NE,Chemical Engineering,PWI
2001,Norman,Loney,Norman Loney,New Jersey Institute of Technology,Associate Professor,NE,Environmental Engineering,PWI
2001,Howard,Matthew,Howard Matthew,Wayne State University,Associate Professor,MW,Chemical Engineering,PWI
2001,Howard,Matthew,Howard Matthew,Wayne State University,Associate Professor,MW,Material Science Engineering,PWI


In [11]:
type_part_df = final_df %>%
    group_by(Year_of_Attendance, Type) %>%
    summarize(n = n()) %>%
    arrange(-n, .by_group = TRUE)

head(type_part_df)

`summarise()` has grouped output by 'Year_of_Attendance'. You can override
using the `.groups` argument.


Year_of_Attendance,Type,n
<dbl>,<chr>,<int>
2001,PWI,40
2001,HBCU,10
2001,AANAPISI,8
2001,HSI,7
2001,Industry,5
2001,Govt,1


In [12]:
type_inst_df = unique(final_df[,c(1,5,9)]) %>%
    group_by(Year_of_Attendance, Type) %>%
    summarize(n = n()) %>%
    arrange(-n, .by_group = TRUE)

head(type_inst_df)

`summarise()` has grouped output by 'Year_of_Attendance'. You can override
using the `.groups` argument.


Year_of_Attendance,Type,n
<dbl>,<chr>,<int>
2001,PWI,29
2001,HBCU,7
2001,HSI,5
2001,Industry,4
2001,AANAPISI,3
2001,Govt,1


In [13]:
# exporting files
write.xlsx(repeat_year_df, paste0(Output,"/", "Q1_Results_", cur_date, ".xlsx"), rowNames = FALSE)
write.xlsx(dept_counts_df, paste0(Output,"/", "Q2_Results_", cur_date, ".xlsx"), rowNames = FALSE)
write.xlsx(regional_part_df, paste0(Output,"/", "Q3a_Results_", cur_date, ".xlsx"), rowNames = FALSE)
write.xlsx(regional_inst_df, paste0(Output,"/", "Q3b_Results_", cur_date, ".xlsx"), rowNames = FALSE)
write.xlsx(type_part_df, paste0(Output,"/", "Q4a_Results_", cur_date, ".xlsx"), rowNames = FALSE)
write.xlsx(type_inst_df, paste0(Output,"/", "Q4b_Results_", cur_date, ".xlsx"), rowNames = FALSE)